# 📚 RAG 시스템 완전 정복: 을GPT 1주차 

을GPT ai및rag팀을 위한 노트북입니다.

공유를 금지합니다. 

## 🎯 학습 목표
이 노트북에서는 다음과 같은 내용을 학습합니다:
- RAG 시스템의 기본 구조와 작동 원리
- 문서 로딩, 분할, 임베딩, 벡터 저장소 생성 과정
- LangChain을 활용한 질의응답 시스템 구축
- RAG 시스템 성능 개선 방법들

In [1]:
from langchain_community.document_loaders import DirectoryLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.chat_models import ChatOllama
from langchain.chains import RetrievalQA

# # 1. 문서 불러오기 (지금 있는 파일)
# loader = DirectoryLoader(
#     path="data",              # 이 폴더 안의
#     glob="**/*.txt",                      # 모든 .txt 파일
#     loader_cls=TextLoader,                # 파일 하나당 TextLoader 사용
#     loader_kwargs={"encoding": "utf-8"}
# )

loader = TextLoader("data/을지대_학업성적처리규정.txt", encoding="utf-8")

documents = loader.load()

# 🔧 1단계: 필요한 라이브러리 Import 및 문서 로딩

RAG 시스템의 첫 번째 단계는 **지식 베이스가 될 문서를 불러오는 것**입니다. 
- LLM은 훈련 데이터에 없는 최신 정보나 특정 도메인 지식을 모릅니다
- 우리의 문서(을지대학교 학업성적처리규정)를 LLM이 참조할 수 있도록 해야 합니다

---

In [2]:
# 2. 문서 쪼개기
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
docs = splitter.split_documents(documents)

# ✂️ 2단계: 문서 분할

## 🤔 왜 분할할까?
- LLM은 처리 길이 제한이 있음  
- 작은 단위로 나누면 검색 정확도↑  
- 질문과 관련된 정보만 추출 가능

## ⚙️ 설정값
- `chunk_size=500`: 한 조각 최대 길이  
- `chunk_overlap=50`: 청크 간 겹침 (문맥 유지)

> 💡 너무 크면 불필요, 너무 작으면 문맥 부족


In [3]:
# 3. 임베딩 모델 (Ollama 사용)
# ollama pull bge-m3
embedding = OllamaEmbeddings(model="bge-m3")  

# 🔢 3단계: 임베딩 모델 설정

## 🎯 임베딩이란?
텍스트를 숫자 벡터로 바꿔 컴퓨터가 의미를 이해하게 하는 과정  
(예: "개" ≈ "강아지")

## 🛠️ bge-m3 모델
- BGE: 중국 BAAI 개발  
- M3: 다국어·다기능·다층 의미 처리  
- 한국어 성능 우수

## 💻 Ollama 사용 이유
- 로컬 실행 (인터넷 불필요)  
- 개인정보 보호  
- API 비용 없음


In [4]:
# 4. Chroma 벡터스토어에 저장
# 이 노트북은 실행 시 시간이 걸릴 수 있습니다. 

vectordb = Chroma.from_documents(
    docs,
    embedding=embedding,
    persist_directory="example_code/chroma_grade_rules"
)
vectordb.persist()

C:\Users\geon1\AppData\Local\Temp\ipykernel_31104\3811263935.py:9: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectordb.persist()


# 🗄️ 4단계: 벡터 DB 생성

## 📊 벡터 DB란?
임베딩 벡터를 저장해 유사도 검색을 빠르게 수행하는 DB  
(질문 → 벡터 → 비슷한 문서 찾기)

## 🌟 Chroma 특징
- 오픈소스, 설치 간단  
- 로컬 저장 → 개인정보 보호  
- 빠른 검색 성능

In [5]:
# 5. LLM
# ollama run joonoh/HyperCLOVAX-SEED-Text-Instruct-1.5B
llm = ChatOllama(model="joonoh/HyperCLOVAX-SEED-Text-Instruct-1.5B")

# 🤖 5단계: LLM 설정

## 🧩 LLM 역할
검색된 문서를 바탕으로 자연스러운 답변 생성

### 🔄 RAG 흐름
1. 관련 문서 검색  
2. 질문 + 문서를 LLM에 입력  
3. 문서 기반 답변 생성

## 🏠 HyperCLOVAX-SEED
- 네이버 한국어 특화 모델  
- 1.5B 파라미터, Instruct형  
- 한국어 문서 이해에 강함

## ⚡ 로컬 실행 장점
- 빠른 응답, API 제한 없음  
- 데이터 유출 걱정 없음


In [6]:
# 6. RetrievalQA 구성
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vectordb.as_retriever(search_kwargs={"k": 3}),
    return_source_documents=True
)

# 🔗 6단계: RAG 체인 구성

## ⛓️ RAG 체인이란?
질문 → 임베딩 → 검색 → LLM 답변까지 연결된 자동화 흐름

### 🔄 동작 순서
1. 질문 입력  
2. 임베딩 변환  
3. 관련 문서 3개 검색 (k=3)  
4. 문서 + 질문 → LLM  
5. 답변 생성 → 반환

## ⚙️ RetrievalQA 주요 옵션
- `chain_type`: 문서 처리 방식  
- `retriever`: 검색 방식  
- `k=3`: 유사 문서 수  
- `return_source_documents=True`: 참고 문서 포함

> 💡 k=3은 정보량과 정확성 사이의 적절한 균형


In [7]:
# 7. 예시 질의
query = "재수강은 최대 몇 학점까지 가능한가요?"
result = qa_chain.invoke(query)

# 8. 출력
print("📌 답변:")
print(result["result"])

print("\n📚 참고 문서 (요약):")
for doc in result["source_documents"]:
    print("-", doc.page_content.strip().split("\n")[0])

📌 답변:
정해진 학점 미달로 졸업이 보류된 자는 학점 미취득 과목에 한하여 재수강을 신청하여야 하며, 한 학기에 학점취소를 위한 재수강신청은 6학점 이내에 한하여 허용됩니다. (단, 2017학년도 신입생부터는 재학연한 이내 총 24학점까지 신청가능하며, 한 학기당 2과목으로 제한합니다.)

📚 참고 문서 (요약):
- ② 편입학생에 대하여는 전적 대학에서 이수한 교과목 및 학점을 심사하여 본 대학에
- ② 편입학생에 대하여는 전적 대학에서 이수한 교과목 및 학점을 심사하여 본 대학에
- ② 편입학생에 대하여는 전적 대학에서 이수한 교과목 및 학점을 심사하여 본 대학에


# 🧪 7단계: RAG 시스템 테스트

## 🎯 테스트 예시
**"재수강은 최대 몇 학점까지 가능한가요?"**  
→ 을지대 학사 규정에서 정확한 숫자 추출 가능 여부 확인

## 📊 결과 구조
- `result`: 최종 답변  
- `source_documents`: 참고 문서


# RAG 시스템 개선법

답변 정확도를 높이기 위한 주요 전략:

1. **의미 기반 텍스트 분할**  
2. **유사도 점수 필터링으로 검색 향상**  
3. **문맥 활용한 프롬프트 개선**  
4. **키워드 + 의미 결합한 하이브리드 검색**  
5. **소스 문서와 일치 여부로 답변 검증**


In [8]:
## 개선 방법 1: 더 나은 텍스트 분할

from langchain.text_splitter import CharacterTextSplitter
import re

def smart_text_splitter(documents):
    """의미 단위로 문서를 분할하는 개선된 함수"""
    # 한국어 문서에 맞는 구분자 설정
    korean_splitter = RecursiveCharacterTextSplitter(
        chunk_size=300,  # 더 작은 청크 크기
        chunk_overlap=100,  # 더 큰 오버랩
        separators=[
            "\n\n",  # 단락 구분
            "\n",    # 줄바꿈
            "。",     # 마침표
            ".",
            " ",     # 공백
            ""
        ],
        keep_separator=True
    )
    
    return korean_splitter.split_documents(documents)

# 개선된 분할 적용
improved_docs = smart_text_splitter(documents)
print(f"기존 문서 수: {len(docs)}")
print(f"개선된 문서 수: {len(improved_docs)}")
print(f"\n첫 번째 청크 예시:")
print(improved_docs[0].page_content[:200] + "...")

기존 문서 수: 10
개선된 문서 수: 20

첫 번째 청크 예시:
학업성적처리규정
제정 2007. 3. 1.
개정 2015. 3. 1.
개정 2017. 3. 1.
개정 2017. 9. 1.
개정 2018. 9. 1.
개정 2020. 3. 1.
개정 2022. 2. 1.
개정 2023. 3. 1.
개정 2024. 3. 1.
개정 2024. 4. 1.
개정 2025. 1.15.
제1조(목적) 이 규정은 을지대학교(이하 “본교...


# 🚀 RAG 성능 개선 시작!

기본 시스템보다 더 정확한 답변을 위해 개선 진행

## 🔧 개선 1: 스마트한 텍스트 분할

### 🤔 기존 문제
- 글자 수 기준 분할 → 의미 단절  
- 중요한 정보가 청크 간에 분리됨

### ✨ 개선 전략
1. 청크 크기 500 → 300  
2. 오버랩 50 → 100  
3. 한국어 맞춤 구분자 사용

### 📊 구분자 우선순위
1. `\n\n` (단락)  
2. `\n` (줄바꿈)  
3. `。`, `.` (문장 끝)  
4. ` ` (공백)


In [9]:
## 개선 방법 2: 새로운 벡터스토어 구성

# 개선된 문서로 새로운 벡터스토어 생성
improved_vectordb = Chroma.from_documents(
    improved_docs,
    embedding=embedding,
    persist_directory="example_code/chroma_grade_rules_improved"
)
improved_vectordb.persist()

print("개선된 벡터스토어가 생성되었습니다.")

개선된 벡터스토어가 생성되었습니다.


# 🔧 개선 방법 2: 새로운 벡터스토어 구성

## 🆕 왜 새로운 벡터스토어를 만들까요?
개선된 텍스트 분할 방식으로 나뉜 문서들을 저장하기 위해 **별도의 벡터스토어**를 생성합니다.

### 📂 저장 경로 구분:
- **기존**: `chroma_grade_rules` - 원본 분할 방식
- **개선**: `chroma_grade_rules_improved` - 개선된 분할 방식

### 💡 분리 저장의 이점:
1. **성능 비교 가능**: 원본과 개선 버전 직접 비교
2. **롤백 가능**: 문제시 원본 버전으로 복구
3. **실험 환경**: 다양한 설정 테스트 가능

### ⚡ 벡터화 과정:
1. 개선된 문서 청크들을 임베딩 모델로 벡터화
2. Chroma 데이터베이스에 저장
3. 디스크에 영구 저장 (persist)

---

In [10]:
## 개선 방법 3: 커스텀 프롬프트 템플릿

from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

# 더 구체적인 프롬프트 템플릿 생성
custom_prompt = PromptTemplate(
    template="""당신은 을지대학교 학업성적처리규정 전문가입니다. 주어진 문서를 바탕으로 정확하고 구체적인 답변을 제공해주세요.

문서 내용:
{context}

질문: {question}

답변 가이드라인:
1. 문서에 명시된 정확한 정보만 사용하세요
2. 학점, 기간, 조건 등 숫자 정보는 정확히 인용하세요
3. 관련 조항이나 예외사항이 있다면 함께 언급하세요
4. 확실하지 않은 정보는 "문서에서 명확히 확인할 수 없습니다"라고 말하세요

답변:""",
    input_variables=["context", "question"]
)

# 검색 결과 포맷팅 함수
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# 개선된 RAG 체인 생성
improved_rag_chain = (
    {"context": improved_vectordb.as_retriever(search_kwargs={"k": 5}) | format_docs, 
     "question": RunnablePassthrough()}
    | custom_prompt
    | llm
    | StrOutputParser()
)

print("커스텀 프롬프트 템플릿이 적용된 RAG 체인이 생성되었습니다.")

커스텀 프롬프트 템플릿이 적용된 RAG 체인이 생성되었습니다.


# 🔧 개선 방법 3: 커스텀 프롬프트 템플릿

## 📝 프롬프트 엔지니어링의 중요성
프롬프트는 LLM에게 **"어떻게 답변해야 하는지"** 알려주는 지시서입니다. 좋은 프롬프트는 답변 품질을 크게 향상시킵니다.

In [11]:
## 개선 방법 4: 유사도 점수 기반 필터링

def enhanced_retrieval(query, vectorstore, threshold=0.5, k=5):
    """유사도 점수를 기반으로 한 향상된 검색"""
    # similarity_search_with_score를 사용하여 점수와 함께 검색
    docs_with_scores = vectorstore.similarity_search_with_score(query, k=k*2)
    
    # 임계값보다 높은 점수의 문서만 필터링
    filtered_docs = [doc for doc, score in docs_with_scores if score > threshold]
    
    # 상위 k개 문서만 반환
    return filtered_docs[:k]

# 테스트를 위한 함수
def test_enhanced_qa(query, vectorstore, custom_prompt, llm):
    """개선된 QA 시스템 테스트"""
    # 향상된 검색 수행
    relevant_docs = enhanced_retrieval(query, vectorstore)
    
    if not relevant_docs:
        return "관련 문서를 찾을 수 없습니다. 질문을 다시 확인해주세요."
    
    # 문서 내용 포맷팅
    context = "\n\n".join([doc.page_content for doc in relevant_docs])
    
    # 프롬프트에 입력
    prompt_input = custom_prompt.format(context=context, question=query)
    
    # LLM 호출
    response = llm.invoke(prompt_input)
    
    return response.content if hasattr(response, 'content') else str(response)

print("향상된 검색 함수가 준비되었습니다.")

향상된 검색 함수가 준비되었습니다.


# 🔧 개선 방법 4: 유사도 점수 기반 필터링

## 📊 유사도 점수란?
벡터 간의 **거리(유사성)**을 0~1 사이의 숫자로 나타낸 것입니다.
- **점수가 낮을수록** = 더 유사함
- **점수가 높을수록** = 덜 유사함

### 🤔 왜 필터링이 필요할까요?
기본 RAG는 **무조건 k개의 문서**를 가져옵니다. 하지만:
- 질문과 관련성이 낮은 문서도 포함될 수 있음
- 관련성 낮은 문서가 답변을 혼란스럽게 만들 수 있음

### ✨ 개선된 검색 전략:
1. **더 많이 검색** (k×2): 후보군 확장
2. **점수로 필터링**: 임계값보다 좋은 문서만 선택
3. **상위 k개만 선택**: 최종적으로 필요한 개수만 사용

### 🎛️ 임계값(threshold) 설정:
- **0.5**: 보통 수준 (너무 관련 없는 문서 제외)
- **0.3**: 엄격한 수준 (매우 관련 있는 문서만)
- **0.8**: 관대한 수준 (웬만한 문서 모두 포함)

### 🧪 실험적 접근:
다양한 임계값을 테스트해서 최적의 값을 찾아보겠습니다!

---

In [12]:
## 개선 방법 5: 실제 테스트 및 비교

def compare_systems(query):
    """기존 시스템과 개선된 시스템 비교"""
    print(f"🔍 질문: {query}")
    print("="*50)
    
    # 기존 시스템
    original_result = qa_chain.invoke(query)
    print("📋 기존 시스템 답변:")
    print(original_result["result"])
    print()
    
    # 개선된 시스템
    improved_answer = improved_rag_chain.invoke(query)
    print("✨ 개선된 시스템 답변:")
    print(improved_answer)
    print("\n" + "="*50 + "\n")


In [13]:
compare_systems('재수강은 몇학점까지 가능한가요? ')

🔍 질문: 재수강은 몇학점까지 가능한가요? 
📋 기존 시스템 답변:
한 학기에 학점취소를 위한 재수강신청은 6학점 이내에 한하여 허용되므로 3과목을 초과할 수 없습니다. 단, 2017학년도 신입생부터는 재학연한 이내 총 24학점까지 신청가능하며 한 학기당 2과목으로 제한됩니다.

✨ 개선된 시스템 답변:
재수강은 한 학기에 6학점 이내로 가능합니다. 이는 문서에 명시된 정보에 근거합니다. 단, 2017학년도 신입생부터는 재학연한이 도래하는 경우 총 24학점까지 신청할 수 있습니다. 또한, 한 학기당 최대 두 과목까지만 제한됩니다.




In [14]:
compare_systems('A등급은 몇 퍼센트 인가요?')

🔍 질문: A등급은 몇 퍼센트 인가요?
📋 기존 시스템 답변:
A등급(A+, A)은 0∼40%입니다.

✨ 개선된 시스템 답변:
문서에 명시된 정보에 따르면 A등급은 0%부터 40%까지 분포 비율에 해당합니다.




In [15]:
compare_systems('절대평가를 실시할 수 있는 과목은 어떤 과목인가요?')

🔍 질문: 절대평가를 실시할 수 있는 과목은 어떤 과목인가요?
📋 기존 시스템 답변:
교직교과목, P/NP 교과목, 특수교육방법교과목, 소규모교과목, 실험ㆍ실습ㆍ실기교과목의 경우에 절대평가를 실시할 수 있습니다.

✨ 개선된 시스템 답변:
문서에 따르면 절대평가를 실시할 수 있는 과목은 교직교과목, P/NP 교과목, 특수교육방법교과목, 소규모교과목, 실험ㆍ실습ㆍ실기교과목입니다.




In [16]:
#문서에 없는 내용을 물어 볼 경우 
compare_systems('졸업 평점은 어떻게 계산하나요?')

🔍 질문: 졸업 평점은 어떻게 계산하나요?
📋 기존 시스템 답변:
저는 졸업 평점을 계산하는 방법에 대한 정보를 가지고 있지 않습니다. 이 정보는 대학이나 학교의 정책에 따라 다를 수 있으므로, 해당 학교나 대학의 공식 웹사이트나 학과에서 정확한 정보를 확인하는 것이 좋습니다.

✨ 개선된 시스템 답변:
죄송합니다만, 졸업 평점에 대한 구체적인 계산을 위한 정보를 문서에서 찾을 수 없습니다. 일반적으로 학기별 평점 평균을 계산한 후, 해당 학기 총 취득 학점이 일정 기준 이상인 경우 그 성적을 우선적으로 고려하여 졸업 평점을 산정하는 것으로 알려져 있습니다.

이 외 더 자세한 사항은 을지대학교의 학무팀이나 학과 담당자에게 직접 문의하시는 것을 추천드립니다.




In [17]:
# improved_rag_chain를 활용한 개별 질의 및 답변

# 1. 재수강은 몇학점까지 가능한가요?
question1 = "재수강은 몇학점까지 가능한가요?"
answer1 = improved_rag_chain.invoke(question1)
print(f"❓ 질문: {question1}")
print(f"💡 답변: {answer1}")

# 2. A등급은 몇 퍼센트 인가요?
question2 = "A등급은 몇 퍼센트 인가요?"
answer2 = improved_rag_chain.invoke(question2)
print(f"\n❓ 질문: {question2}")
print(f"💡 답변: {answer2}")

# 3. 절대평가를 실시할 수 있는 과목은 어떤 과목인가요?
question3 = "절대평가를 실시할 수 있는 과목은 어떤 과목인가요?"
answer3 = improved_rag_chain.invoke(question3)
print(f"\n❓ 질문: {question3}")
print(f"💡 답변: {answer3}")

❓ 질문: 재수강은 몇학점까지 가능한가요?
💡 답변: 문서에 따르면, 재수강은 한 학기에 6학점 이내로 가능합니다. 단, 2017학년도 신입생부터는 재학연한 이내 총 24학점까지 신청가능합니다. 또한, 한 학기당 최대 두 과목으로 제한됩니다. (개정 2017.9.1.)

❓ 질문: A등급은 몇 퍼센트 인가요?
💡 답변: 문서에 따르면 A등급은 0∼40%의 성적 분포 비율을 가집니다.

❓ 질문: 절대평가를 실시할 수 있는 과목은 어떤 과목인가요?
💡 답변: 네, 절대평가를 실시할 수 있는 과목은 교직교과목, P/NP 교과목, 특수교육방법교과목, 소규모교과목, 실험ㆍ실습ㆍ실기교과목의 경우에는 절대평가를 실시할 수 있습니다. 이는 2017년 3월 1일에 개정된 academic performance handling regulations에 명시되어 있으며, 해당 규정은 2022년 2월 1일과 2024년 4월 1일에 추가로 수정되었습니다.


감사합니다. 